In [1]:
import cv2
import numpy as np
from PIL import Image

In [2]:
original_ortho = '/home/caluckal/Desktop/Github/upscaler/validation/original_Ortho.png'
altered_ortho = '/home/caluckal/Desktop/Github/upscaler/validation/downscaled_upscaled_ISR_Ortho.png'

In [4]:
# og_img = cv2.imread(original_ortho)

In [3]:
alt_img =cv2.imread(altered_ortho)

In [4]:
#Corner Detection
def corner(image,name):
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray,2,3,0.04)

    #result is dilated for marking the corners, not important
    dst = cv2.dilate(dst,None)

    # Threshold for an optimal value, it may vary depending on the image.
    image[dst>0.01*dst.max()]=[0,0,255]

    cv2.imwrite('harris_{}.png'.format(name),image)

In [5]:
# Edge Detection
def edge(img,name):
    # Convert to graycsale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Blur the image for better edge detection
    img_blur = cv2.GaussianBlur(img_gray, (3,3), 0) 
    # Sobel Edge Detection
    sobelx = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
    cv2.imwrite('sobelX_{}.png'.format(name),sobelx)
    sobely = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
    cv2.imwrite('sobelY_{}.png'.format(name),sobely)
    sobelxy = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection
    cv2.imwrite('sobelXY_{}.png'.format(name),sobelxy)
    edges = cv2.Canny(image=img_blur, threshold1=100, threshold2=200) # Canny Edge Detection
    # Display Canny Edge Detection Image
    cv2.imwrite('canny_{}.png'.format(name),edges)


In [2]:
keypoints = 0
def sift(image,name):
    global keypoints
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    kp = sift.detect(gray,None)
    keypoints+=len(kp)
    img=cv2.drawKeypoints(gray,kp,image)
    cv2.imwrite(name[:-4]+'_new.jpg',img)

In [9]:
# corner(og_img,'og')

In [6]:
corner(alt_img,'alt')

In [11]:
# edge(og_img,'og')


In [7]:
edge(alt_img,'alt')

In [9]:
import os
folder = '/home/caluckal/Desktop/Github/upscaler/validation/IP_testing/ISR/sift_results/og_parts'
img_list = [x for x in os.listdir(folder) if x.endswith('.jpg')]

for x in img_list:
    image = cv2.imread(folder+'/'+x)
    sift(image,'/home/caluckal/Desktop/Github/upscaler/validation/IP_testing/ISR/sift_results/og_sift/'+x)

print(keypoints)
keypoints = 0

4319033


In [ ]:
import os
folder = '/home/caluckal/Desktop/Github/upscaler/validation/IP_testing/ISR/sift_results/alt_parts'
img_list = [x for x in os.listdir(folder) if x.endswith('.jpg')]

for x in img_list:
    image = cv2.imread(folder+'/'+x)
    sift(image,'/home/caluckal/Desktop/Github/upscaler/validation/IP_testing/ISR/sift_results/alt_sift/'+x)

print(keypoints)
keypoints = 0

In [15]:
def sharpen(image):
    kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
    image_sharp = cv2.filter2D(src=image, ddepth=-1, kernel=kernel)
    return image_sharp

import os
folder = '/home/caluckal/Desktop/Github/upscaler/validation/IP_testing/ISR/sift_results/alt_parts'

img_list = [x for x in os.listdir(folder) if x.endswith('.jpg')]

for x in img_list:
    image = cv2.imread(folder+'/'+x)
    sift(image,'/home/caluckal/Desktop/Github/upscaler/validation/IP_testing/ISR/sift_results/alt_sift/'+x)


print(keypoints)
keypoints = 0

In [16]:
def sharpen(image):
    kernel = np.array([[0, -2, 0],
                   [-2, 9,-2],
                   [0, -2, 0]])
    image_sharp = cv2.filter2D(src=image, ddepth=-1, kernel=kernel)
    return image_sharp

import os
folder = 'sift/alt_parts'
img_list = [x for x in os.listdir(folder) if x.endswith('.jpg')]

for x in img_list:
    image = cv2.imread(folder+'/'+x)
    sift(sharpen(image),'sift/alt_sift/'+x)

print(keypoints)
keypoints = 0

In [3]:
def match(img1,img2,name):
#load images
    # Initiate SIFT detector
    img1 = cv2.imread(img1)
    img2 = cv2.imread(img2)
    sift = cv2.SIFT()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)

    # BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1,des2, k=2)
    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])

    img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,flags=2)
    cv2.imwrite(name+'.jpg',img3)
    return good

In [5]:
import os
import cv2

good_list = []

og_folder = '/home/caluckal/Desktop/Github/upscaler/validation/IP_testing/ISR/sift_results/og_parts'
alt_folder = '/home/caluckal/Desktop/Github/upscaler/validation/IP_testing/ISR/sift_results/alt_parts'


og_list = sorted([x for x in os.listdir(og_folder) if x.endswith('.jpg')])[0:1]
alt_list = sorted([x for x in os.listdir(alt_folder) if x.endswith('.jpg')])

match_list = []
name_list = []
for x in og_list:
    names = x.split('_')
    name = names[2]+'_'+names[3]
    if 'alt_parts_'+name+'_.jpg' in alt_list:
        match_list.append([x,'alt_parts_'+name+'_.jpg'])

match_file = open('/home/caluckal/Desktop/Github/upscaler/validation/IP_testing/ISR/sift_results/match_list.txt','w+')

for x in match_list:
    names = x[0].split('_')
    name = names[2]+'_'+names[3]
    good_list.append(match(og_folder+'/'+x[0],alt_folder+'/'+x[1],'/home/caluckal/Desktop/Github/upscaler/validation/IP_testing/ISR/sift_results/match/'+name))
    name_list.append(name)

for x,y in zip(good_list,name_list):
    print(y,':',len(x))
    match_file.write((y,':',len(x)+'\n'))



In [1]:
srgan_res = open('/home/caluckal/Desktop/Github/upscaler/validation/IP_testing/SRGAN/sift_results/match_file.txt')
isr_res = open('/home/caluckal/Desktop/Github/upscaler/validation/IP_testing/ISR/sift_results/match_list.txt')
pil_res = open('/home/caluckal/Desktop/Github/upscaler/validation/IP_testing/PIL/sift_results/match/match_list.txt')

srgan_sum = 0
isr_sum = 0
pil_sum = 0
for x in srgan_res.readlines():
    srgan_sum+= int(x.split(':')[-1])

print("SRGAN txt:",srgan_sum)

for x in isr_res.readlines():
    isr_sum+= int(x.split(':')[-1])

print("ISR txt:",isr_sum)

for x in pil_res.readlines():
    pil_sum+= int(x.split(':')[-1])

print("ISR txt:",pil_sum)

SRGAN txt: 137304
ISR txt: 1518960
ISR txt: 1028102
